# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Feb 9 2023 9:59AM,257027,20,ATR-I000098,"HVL, LLC dba Atrium Innovations",Released
1,Feb 9 2023 9:56AM,257026,10,Eme-108475,Emerginnova,Released
2,Feb 9 2023 9:56AM,257026,10,Enova-11336,Emerginnova,Released
3,Feb 9 2023 9:54AM,257024,10,PRF-52374,Bio-PRF,Released
4,Feb 9 2023 9:54AM,257024,10,PRF-52379,Bio-PRF,Released
5,Feb 9 2023 9:54AM,257024,10,PRF-52383,Bio-PRF,Released
6,Feb 9 2023 9:54AM,257024,10,PRF-52402,Bio-PRF,Released
7,Feb 9 2023 9:54AM,257024,10,PRF-52431,Bio-PRF,Released
8,Feb 9 2023 9:54AM,257024,10,PRF-52436,Bio-PRF,Released
9,Feb 9 2023 9:45AM,256934,15,30013807,"Alliance Pharma, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
41,257023,Released,1
42,257024,Released,6
43,257025,Released,1
44,257026,Released,2
45,257027,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
257023,NaN,NaN,1.0
257024,NaN,NaN,6.0
257025,NaN,NaN,1.0
257026,NaN,NaN,2.0
257027,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
256826,46.0,0.0,66.0
256908,3.0,0.0,1.0
256917,1.0,2.0,4.0
256931,0.0,0.0,1.0
256934,0.0,0.0,43.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
256826,46,0,66
256908,3,0,1
256917,1,2,4
256931,0,0,1
256934,0,0,43


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,256826,46,0,66
1,256908,3,0,1
2,256917,1,2,4
3,256931,0,0,1
4,256934,0,0,43


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,256826,46,,66
1,256908,3,,1
2,256917,1,2,4
3,256931,,,1
4,256934,,,43


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Feb 9 2023 9:59AM,257027,20,"HVL, LLC dba Atrium Innovations"
1,Feb 9 2023 9:56AM,257026,10,Emerginnova
3,Feb 9 2023 9:54AM,257024,10,Bio-PRF
9,Feb 9 2023 9:45AM,256934,15,"Alliance Pharma, Inc."
52,Feb 9 2023 9:45AM,257025,19,"GCH Granules USA, Inc."
53,Feb 9 2023 8:58AM,257020,16,"SHL Pharma, LLC"
54,Feb 9 2023 8:57AM,257023,12,"Uniderm USA, Inc"
55,Feb 9 2023 8:55AM,257022,10,"Digital Brands, LLC"
66,Feb 9 2023 8:53AM,257021,12,"SD Head USA, LLC"
67,Feb 9 2023 8:32AM,257014,10,ISDIN Corporation


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Feb 9 2023 9:59AM,257027,20,"HVL, LLC dba Atrium Innovations",,,1
1,Feb 9 2023 9:56AM,257026,10,Emerginnova,,,2
2,Feb 9 2023 9:54AM,257024,10,Bio-PRF,,,6
3,Feb 9 2023 9:45AM,256934,15,"Alliance Pharma, Inc.",,,43
4,Feb 9 2023 9:45AM,257025,19,"GCH Granules USA, Inc.",,,1
5,Feb 9 2023 8:58AM,257020,16,"SHL Pharma, LLC",,1,
6,Feb 9 2023 8:57AM,257023,12,"Uniderm USA, Inc",,,1
7,Feb 9 2023 8:55AM,257022,10,"Digital Brands, LLC",,,11
8,Feb 9 2023 8:53AM,257021,12,"SD Head USA, LLC",,,1
9,Feb 9 2023 8:32AM,257014,10,ISDIN Corporation,,,70


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 9 2023 9:59AM,257027,20,"HVL, LLC dba Atrium Innovations",1,,
1,Feb 9 2023 9:56AM,257026,10,Emerginnova,2,,
2,Feb 9 2023 9:54AM,257024,10,Bio-PRF,6,,
3,Feb 9 2023 9:45AM,256934,15,"Alliance Pharma, Inc.",43,,
4,Feb 9 2023 9:45AM,257025,19,"GCH Granules USA, Inc.",1,,
5,Feb 9 2023 8:58AM,257020,16,"SHL Pharma, LLC",,1,
6,Feb 9 2023 8:57AM,257023,12,"Uniderm USA, Inc",1,,
7,Feb 9 2023 8:55AM,257022,10,"Digital Brands, LLC",11,,
8,Feb 9 2023 8:53AM,257021,12,"SD Head USA, LLC",1,,
9,Feb 9 2023 8:32AM,257014,10,ISDIN Corporation,70,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 9 2023 9:59AM,257027,20,"HVL, LLC dba Atrium Innovations",1,,
1,Feb 9 2023 9:56AM,257026,10,Emerginnova,2,,
2,Feb 9 2023 9:54AM,257024,10,Bio-PRF,6,,
3,Feb 9 2023 9:45AM,256934,15,"Alliance Pharma, Inc.",43,,
4,Feb 9 2023 9:45AM,257025,19,"GCH Granules USA, Inc.",1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 9 2023 9:59AM,257027,20,"HVL, LLC dba Atrium Innovations",1.0,NaN,NaN
1,Feb 9 2023 9:56AM,257026,10,Emerginnova,2.0,NaN,NaN
2,Feb 9 2023 9:54AM,257024,10,Bio-PRF,6.0,NaN,NaN
3,Feb 9 2023 9:45AM,256934,15,"Alliance Pharma, Inc.",43.0,NaN,NaN
4,Feb 9 2023 9:45AM,257025,19,"GCH Granules USA, Inc.",1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 9 2023 9:59AM,257027,20,"HVL, LLC dba Atrium Innovations",1.0,0.0,0.0
1,Feb 9 2023 9:56AM,257026,10,Emerginnova,2.0,0.0,0.0
2,Feb 9 2023 9:54AM,257024,10,Bio-PRF,6.0,0.0,0.0
3,Feb 9 2023 9:45AM,256934,15,"Alliance Pharma, Inc.",43.0,0.0,0.0
4,Feb 9 2023 9:45AM,257025,19,"GCH Granules USA, Inc.",1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,3854760,187.0,3.0,4.0
12,771046,3.0,0.0,0.0
15,770691,110.0,0.0,46.0
16,257020,0.0,1.0,0.0
19,2569827,65.0,0.0,1.0
20,514034,2.0,0.0,0.0
21,1027946,4.0,0.0,0.0
22,513991,2.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,3854760,187.0,3.0,4.0
1,12,771046,3.0,0.0,0.0
2,15,770691,110.0,0.0,46.0
3,16,257020,0.0,1.0,0.0
4,19,2569827,65.0,0.0,1.0
5,20,514034,2.0,0.0,0.0
6,21,1027946,4.0,0.0,0.0
7,22,513991,2.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,187.0,3.0,4.0
1,12,3.0,0.0,0.0
2,15,110.0,0.0,46.0
3,16,0.0,1.0,0.0
4,19,65.0,0.0,1.0
5,20,2.0,0.0,0.0
6,21,4.0,0.0,0.0
7,22,2.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,187.0
1,12,Released,3.0
2,15,Released,110.0
3,16,Released,0.0
4,19,Released,65.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,19,20,21,22
Status,,,,,,,,
Completed,4.0,0.0,46.0,0.0,1.0,0.0,0.0,0.0
Executing,3.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
Released,187.0,3.0,110.0,0.0,65.0,2.0,4.0,2.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,19,20,21,22
0,Completed,4.0,0.0,46.0,0.0,1.0,0.0,0.0,0.0
1,Executing,3.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,Released,187.0,3.0,110.0,0.0,65.0,2.0,4.0,2.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,19,20,21,22
0,Completed,4.0,0.0,46.0,0.0,1.0,0.0,0.0,0.0
1,Executing,3.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,Released,187.0,3.0,110.0,0.0,65.0,2.0,4.0,2.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()